In [3]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LogisticRegression as lr
import graphviz
import nltk
from nltk.chunk import *
from nltk.chunk.util import *
from nltk.chunk.regexp import *
from nltk import Tree

In [2]:
def wordToVec(sentences):
    vec = np.zeros(len(sentences))
    word = [s.split(' ') for s in sentences]
    #print(word)
    model = Word2Vec(word, min_count=1)
    #print(model.wv.get_vector(word[0][0]).shape)
    for i in range(len(word)):
        temp = np.zeros(100)
        for j in range(len(word[i])):
            temp += model.wv.get_vector(word[i][j])
        # take average of the sentences
        vec[i] = np.sum(temp)
    #print(vec)
    return vec

In [4]:
def ruleBasedSearch(sentenceList):
    # return new dataframe, colName: to be, other verb, verb, punctuation, pronoun
    newPd = pd.DataFrame(columns=['toBe','defVerb','verb','punctuation','pronoun'])
    for sen in sentenceList:
        #print(sen)
        result = [0]*5
        # tobe
        reTOBE = re.compile(r'\b(be|is|are|was|were|being{1})\b', flags=re.IGNORECASE)
        if (reTOBE.findall(sen)!=[]):
            result[0] = 1
        # defVerb
        reDefVerb = re.compile(r'\b(as|define|defines|defined|mean|means|meant|refer|refers|referred{1})\b', flags=re.IGNORECASE)
        if (reDefVerb.findall(sen)!=[]):
            result[1] = 1
        # punctuation
        rePunc = re.compile(r'(:|-|\()')
        if (rePunc.findall(sen)!=[]):
            result[3] = 1
        
        
        tokens = nltk.word_tokenize(sen)
        tagged = nltk.pos_tag(tokens)
        #print(tagged)

        tagged_text_string = " ".join(["{}/{}".format(word,pos) for word, pos in tagged])
        gold_chunked_text = tagstr2tree(tagged_text_string)
        unchunked_text = gold_chunked_text.flatten()
        
        # verb
        chunk_rule = ChunkRule("<VB|VBG|VBD|VBN|VBP|VBZ>", "Chunk verb")
        chunk_parser = RegexpChunkParser([chunk_rule], chunk_label='NP')
        chunked_text = chunk_parser.parse(unchunked_text)
        #print(chunked_text)
        if (chunked_text.height()>=3):
            result[2] = 1
        # pronoun
        chunk_rule1 = ChunkRule("<PRP|PRP\$>", "Chunk pronoun")
        chunk_parser1 = RegexpChunkParser([chunk_rule1], chunk_label='NP')
        chunked_text1 = chunk_parser1.parse(unchunked_text)
        if (chunked_text1.height()>=3):
            result[4] = 1
        
        """
        posTag = ' '.join(pos for _,pos in nltk.pos_tag(nltk.word_tokenize(sen)))
        print(posTag)
        # verb
        reVerb = re.compile(r'(VB?)', flags=re.IGNORECASE)
        if (reVerb.findall(sen)!=[]):
            result[2] = 1
        # pronoun
        rePronoun = re.compile(r'(PRP)', flags=re.IGNORECASE)
        if (rePronoun.findall(sen)!=[]):
            result[4] = 1
        """
        newPd.loc[len(newPd)] = result
    return newPd

#test = ["is not sth","a is defined as sth","i am moving a car","a() sth","in our model"]
#print(ruleBasedSearch(test))
        

  toBe defVerb verb punctuation pronoun
0    1       0    1           0       0
1    1       1    1           0       0
2    0       0    1           0       0
3    0       0    0           1       0
4    0       0    0           0       1


In [51]:
#####
# file strcture:
# 1. sentence: equation with symbol replaced with EQUATION
#              equation without symbol replaced with OBJECT
# 2. section sentence appears: number of section/total section
#             e.g.: Introduction:1/n, Section 3.1 (there are 3.1-3.3):3.33333/n
# 3. sameSubSection: is the symbol selected the same subsection as the sentence, binary 0/1
# 4. sameSection: is the symbol selected the same section as the sentence, binary 0/1
# 5. distanceToSection: how many sections between sentence & symbol
# 6. number of appearance: number of symbol appearance/total number of appearance
# 7. distanceToSymbol: how many sentences between current sentence & symbol
# 8. label: 0 - weak, 1 - medium, 2 - strong

df = pd.read_csv('testset_jenny.csv', names=['sentence','section','sameSubSection','sameSection','distanceToSection'
                                             ,'numAppearance','distanceToSymbol','label'], header=None)
#print(type(df[df.columns[0]]))
""" #word2Vec representation
vec = wordToVec(df[df.columns[0]].tolist())
df.insert(1,"vec",vec,allow_duplicates=True)
"""

vec = ruleBasedSearch(df[df.columns[0]].tolist())
df = pd.concat([df,vec], axis=1)
print(df)

#############
# now the columns are: 'sentence','section','sameSubSection','sameSection','distanceToSection','numAppearance'
#     ,'distanceToSymbol','label','toBe','defVerb','verb','punctuation','pronoun'
#############
 
#X = df[['vec','section','numAppearance']]
#X = df[['section','sameSubSection','sameSection','distanceToSection','distanceToSymbol']]
#X = df[['sameSubSection','sameSection','distanceToSection','distanceToSymbol']]
X = df[['toBe','defVerb','verb','punctuation','pronoun','section','sameSubSection','sameSection','distanceToSection','numAppearance','distanceToSymbol']]
Y = df['label']

                                             sentence   section  \
0   In particular consider the conditional log-pro...  0.222222   
1   With this scoring function in mind, our main f...  0.222222   
2   The distribution of interest, EQUATION, is a c...  0.333333   
3         The full model is: EQUATION, OBJECT, OBJECT  0.370370   
4   The parameters are EQUATION where OBJECT is a ...  0.370370   
5   Once we have defined the local conditional mod...  0.444443   
6                            OBJECT, EQUATION, OBJECT  0.444444   
7   \Require{Parameters EQUATION, beam size OBJECT...  0.444444   
8   We do this by modifying our scoring function t...  0.444444   
9   After training the main neural model, we fix E...  0.444444   
10  where EQUATION is a size of the mini-batch and...  0.600000   
11  The leading term of the number of multiply-add...  0.714286   
12  By the weak law of large numbers, when EQUATIO...  0.600000   
13                           OBJECT, EQUATION, OBJECT  0.60000

In [52]:
# SVM
clf = svm.SVC(kernel='linear')
clf.fit(X,Y)
print("algo coefficients:")
print(clf.coef_)
print("intercept:")
print(clf.intercept_)

algo coefficients:
[[-5.41600090e-01  4.58399910e-01 -7.44459086e-01 -7.44459086e-01
  -1.00000000e+00 -1.33061261e-02  3.40104084e-01  3.40104084e-01
   2.54177924e-01 -4.00611704e-01  1.87201026e-01]
 [-8.66676922e-01  1.00000000e+00 -8.66676922e-01 -1.71383179e-01
   0.00000000e+00 -1.73396725e-01  3.33836269e-01  3.33836269e-01
  -2.91300115e-02 -2.37780885e-01  5.80282503e-01]
 [-6.23476788e-01  1.00000000e+00 -4.44089210e-16  3.76523212e-01
   1.28964281e-01 -1.23887292e-01  1.28964281e-01  1.28964281e-01
  -1.28964281e-01  1.77378568e-01  8.60060733e-01]]
intercept:
[ 0.45918149  0.01970929 -2.08922388]


In [62]:
# predict
print(clf.predict([list(X.iloc[4])]))

[2]


In [53]:
# Multinomial Logit
mlr = lr(multi_class="multinomial", solver="newton-cg")
mlr.fit(X,Y)
print("algo coefficients:")
print(mlr.coef_)
print("intercept:")
print(mlr.intercept_)
print("difference in coefficients:")
print(clf.coef_-mlr.coef_)
print("difference in intercept:")
print(clf.intercept_-mlr.intercept_)

algo coefficients:
[[-4.48583225e-01  3.10902622e-01 -8.21451143e-01 -3.48818405e-01
  -4.64027397e-01 -2.72147493e-02  4.68629335e-01  2.53647512e-01
   1.64359361e-01 -2.13806896e-01  2.25532421e-01]
 [-1.98061115e-01  3.57779187e-01  3.95049388e-01  3.48169638e-01
   4.74558048e-01 -7.73825036e-02 -3.60438712e-01 -6.08508559e-02
  -1.02843539e-01  1.55858566e-01  1.81404935e-01]
 [ 6.46644340e-01 -6.68681809e-01  4.26401755e-01  6.48766071e-04
  -1.05306503e-02  1.04597253e-01 -1.08190622e-01 -1.92796656e-01
  -6.15158224e-02  5.79483296e-02 -4.06937356e-01]]
intercept:
[ 0.67625073 -0.8383667   0.16211597]
difference in coefficients:
[[-0.09301686  0.14749729  0.07699206 -0.39564068 -0.5359726   0.01390862
  -0.12852525  0.08645657  0.08981856 -0.18680481 -0.03833139]
 [-0.66861581  0.64222081 -1.26172631 -0.51955282 -0.47455805 -0.09601422
   0.69427498  0.39468712  0.07371353 -0.39363945  0.39887757]
 [-1.27012113  1.66868181 -0.42640176  0.37587445  0.13949493 -0.22848455
   0.2

In [59]:
# predict
print(mlr.predict([list(X.iloc[3])]))


[1]


In [2]:
# decision tree
dtree = tree.DecisionTreeClassifier()
dtree.fit(X,Y)
#print(dtree.decision_path(X))
dot_data = tree.export_graphviz(dtree, out_file=None,feature_names=list(X.columns), 
                                filled=True,special_characters=True)
graph = graphviz.Source(dot_data)
graph

NameError: name 'tree' is not defined

In [57]:
# predict
print(dtree.predict([list(X.iloc[4])]))


[2]


In [29]:
# test with NLTK

testSen="With this scoring function in mind, to is are our main focus will be on modelling the local conditional distribution: EQUATION."

tokens = nltk.word_tokenize(testSen)
tagged = nltk.pos_tag(tokens)
#print(tagged)

tagged_text_string = " ".join(["{}/{}".format(word,pos) for word, pos in tagged])
gold_chunked_text = tagstr2tree(tagged_text_string)
unchunked_text = gold_chunked_text.flatten()

chunk_rule = ChunkRule("<VBG|VB|VBD>", "Chunk test")
chunk_parser = RegexpChunkParser([chunk_rule], chunk_label='NP')
chunked_text = chunk_parser.parse(unchunked_text)
print(chunked_text)
print(chunked_text.height())

"""
entities = nltk.chunk.ne_chunk(tagged)
print(entities)

if 'PRP' in {pos for _,pos in nltk.pos_tag(nltk.word_tokenize(testSen))}:
    print('true')
#nltk.regexp_tokenize(testSen, r'[\!]\s*', gaps=True)
import re
# t = re.search(r'is|this',testSen)
# t.groups()
regex = re.compile(r'\b(in minds|with|thiss{1})\b', flags=re.IGNORECASE)
print(regex.findall(testSen))
"""

(S
  With/IN
  this/DT
  (NP scoring/VBG)
  function/NN
  in/IN
  mind/NN
  ,/,
  to/TO
  is/VBZ
  are/VBP
  our/PRP$
  main/JJ
  focus/NN
  will/MD
  (NP be/VB)
  on/IN
  (NP modelling/VBG)
  the/DT
  local/JJ
  conditional/JJ
  distribution/NN
  :/:
  EQUATION/NN
  ./.)
3


"\nentities = nltk.chunk.ne_chunk(tagged)\nprint(entities)\n\nif 'PRP' in {pos for _,pos in nltk.pos_tag(nltk.word_tokenize(testSen))}:\n    print('true')\n#nltk.regexp_tokenize(testSen, r'[\\!]\\s*', gaps=True)\nimport re\n# t = re.search(r'is|this',testSen)\n# t.groups()\nregex = re.compile(r'\x08(in minds|with|thiss{1})\x08', flags=re.IGNORECASE)\nprint(regex.findall(testSen))\n"